# Rough Experimentation with merging ICD Codes to Patient Diagnoses
This notebook contains code experimenting with merging ICD codes to patient diagnoses. In particular, the 'inpatient_admissions_qual.csv' file is used to identify the first and second listed discharge diagnoses. The dataframe will be merged again on the ICD code to incorporate the CC/MCC designation for each Dx.

Adjust filepaths to your need.

Additional formatting of the ICD codes and patient dx is needed to fully match each patient to an ICD code.

*Aditya Vijay 7/5/2023*

In [1]:
# Imports
import pandas as pd
import numpy as np
import os
from google.colab import drive, files

In [2]:
# Mount Drive - Adjust folder path
drive.mount('/content/drive')
folder_path = "/content/drive/MyDrive/Colab Notebooks/v-CHAMPS/SyntheticQualityCheck"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Loading CSVs
df_list = []
filenames = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
for filename in filenames:
  df_list.append(pd.read_csv(folder_path + "/" + filename, encoding = "utf-8"))


In [4]:
# List Files
[(i, file) for i,file in enumerate(filenames)]

[(0, 'demographics_static_qual.csv'),
 (1, 'inpatient_specialty_qual.csv'),
 (2, 'immunization_qual.csv'),
 (3, 'conditions_qual.csv'),
 (4, 'measurements_blood_pressure_qual.csv'),
 (5, 'demographics_event_qual.csv'),
 (6, 'procedures_qual.csv'),
 (7, 'medications_ordered_qual.csv'),
 (8, 'lab_results_qual.csv'),
 (9, 'outpatient_visits_qual.csv'),
 (10, 'measurements_qual.csv'),
 (11, 'medications_administered_qual.csv'),
 (12, 'ed_visits_qual.csv'),
 (13, 'inpatient_location_qual.csv'),
 (14, 'inpatient_admissions_qual.csv')]

In [5]:
# Formatting inpatient_admissions DF
inpt_adm = df_list[14]
inpt_adm.rename(columns = {
    "First.listed.discharge.diagnosis.icd10.subcategory": "first_dx",
    "Second.listed.discharge.diagnosis.icd10.subcategory":"second_dx"}, inplace = True)
inpt_adm.first_dx = [dx.lower().replace(",", "") for dx in inpt_adm.first_dx]
inpt_adm.second_dx = [dx.lower().replace(",", "") for dx in inpt_adm.second_dx]
inpt_adm.head()

,Unnamed: 0,X,Internalpatientid,Age.at.admission,Admission.date,Discharge.date,Admitting.unit.service,Discharging.unit.service,Admitting.specialty,Discharging.specialty,first_dx,second_dx,Discharge.disposition,Died.during.admission,Outpatientreferralflag,Serviceconnectedflag,Agentorangeflag,State
0,1,4,100012,55.317020,2004-10-25 08:54:01.0,2004-10-26 08:05:06.0,SURGERY,SURGERY,NEUROSURGERY,GENERAL SURGERY,other and unspecified noninfective gastroenter...,other specified disorders of white blood cells,Regular,NaN,Yes,NaN,NaN,New Mexico
1,2,82,100399,85.706740,2010-03-24 19:31:38.0,2012-11-10 19:50:48.0,NHCU,NHCU,DOMICILIARY,NH HOSPICE,unspecified mental disorder due to known physi...,malignant neoplasm of stomach unspecified,Death without autopsy,NaN,NaN,No,No,Minnesota
2,3,154,100694,83.926120,2016-01-12 23:55:24.0,2016-01-13 20:55:24.0,NON-COUNT,NON-COUNT,SPINAL CORD INJURY,MEDICAL OBSERVATION,abnormal levels of other serum enzymes,other acute ischemic heart diseases,Regular,NaN,Yes,NaN,No,Idaho
3,4,155,100694,88.611203,2020-09-20 09:28:13.0,2020-09-22 11:36:18.0,NON-COUNT,NON-COUNT,SPINAL CORD INJURY,MEDICAL OBSERVATION,viral intestinal infection unspecified,hypo-osmolality and hyponatremia,Regular,NaN,Yes,NaN,No,Idaho
4,5,322,101407,88.925931,2009-05-03 10:15:50.0,2009-05-04 13:05:08.0,MEDICINE,MEDICINE,SPINAL CORD INJURY OBSERVATION,GENERAL(ACUTE MEDICINE),unspecified dementia,hypertensive chronic kidney disease with stage...,Regular,NaN,Yes,NaN,No,Louisiana


In [6]:
# Reading and Formatting ICD10 codes files
icd_codes = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/v-CHAMPS/icd10cm_codes_2018.csv", encoding="utf-8" ,header = None)
icd_codes.fillna("", inplace=True)
icd_codes['description'] = icd_codes[0] + " " + icd_codes[1]
icd_codes['code'] = [code.split(" ")[0] for code in icd_codes['description']]
icd_codes['description'] = [des.split(" ", 1)[1] for des in icd_codes['description']]
icd_codes = icd_codes.drop(columns = [0,1,2,3,4,5])[['code','description']]
icd_codes.description = [desc.lower().strip().replace("  "," ") for desc in icd_codes.description]
icd_codes.head()

,code,description
0,A000,cholera due to vibrio cholerae 01 biovar cholerae
1,A001,cholera due to vibrio cholerae 01 biovar eltor
2,A009,cholera unspecified
3,A0100,typhoid fever unspecified
4,A0101,typhoid meningitis


## Examining ICD Duplicates
The icd_codes DF has multiple duplicates of descriptions, with different codes associated with each one.

In [7]:
# Each duplicate has a different ICD code
icd_codes[icd_codes.duplicated("description")].description.value_counts()

contusion laceration                                    51
collapsed vertebra not elsewhere classified             35
coma scale best motor response                          29
disorder of amniotic fluid and membranes unspecified    27
infection of amniotic sac and membranes unspecified     27
                                                        ..
drug-induced chronic gout right wrist                    1
drug-induced chronic gout unspecified elbow              1
drug-induced chronic gout left elbow                     1
drug-induced chronic gout right elbow                    1
multiple births unspecified                              1
Name: description, Length: 5237, dtype: int64

In [8]:
# There are no duplicates of ICD codes themselves
print(len(icd_codes[icd_codes.duplicated("code")].code.value_counts()))

0


In [9]:
# Writing Duplicates to CSV
# Change file path and remove block quotes to write csv
"""
ICDduplicates = pd.DataFrame(data = {
    "ICD Description":icd_codes[icd_codes.duplicated("description")].description.value_counts().index,
    "Count":icd_codes[icd_codes.duplicated("description")].description.value_counts().values})
ICDduplicates.to_csv("/content/drive/MyDrive/Colab Notebooks/v-CHAMPS/icd_duplicates.csv", index = False)
"""

'\nICDduplicates = pd.DataFrame(data = {\n    "ICD Description":icd_codes[icd_codes.duplicated("description")].description.value_counts().index, \n    "Count":icd_codes[icd_codes.duplicated("description")].description.value_counts().values})\nICDduplicates.to_csv("/content/drive/MyDrive/Colab Notebooks/v-CHAMPS/icd_duplicates.csv", index = False)\n'

## Merging DataFrames

The "inpt_adm" dataframe and "icd_codes" dataframe are merged by matching on the 'first_dx' and 'description' columns.

In [10]:
merged_df = pd.merge(inpt_adm, icd_codes, left_on = 'first_dx', right_on = "description", how = 'left')
merged_df.head(5)

,Unnamed: 0,X,Internalpatientid,Age.at.admission,Admission.date,Discharge.date,Admitting.unit.service,Discharging.unit.service,Admitting.specialty,Discharging.specialty,first_dx,second_dx,Discharge.disposition,Died.during.admission,Outpatientreferralflag,Serviceconnectedflag,Agentorangeflag,State,code,description
0,1,4,100012,55.317020,2004-10-25 08:54:01.0,2004-10-26 08:05:06.0,SURGERY,SURGERY,NEUROSURGERY,GENERAL SURGERY,other and unspecified noninfective gastroenter...,other specified disorders of white blood cells,Regular,NaN,Yes,NaN,NaN,New Mexico,NaN,NaN
1,2,82,100399,85.706740,2010-03-24 19:31:38.0,2012-11-10 19:50:48.0,NHCU,NHCU,DOMICILIARY,NH HOSPICE,unspecified mental disorder due to known physi...,malignant neoplasm of stomach unspecified,Death without autopsy,NaN,NaN,No,No,Minnesota,F09,unspecified mental disorder due to known physi...
2,3,154,100694,83.926120,2016-01-12 23:55:24.0,2016-01-13 20:55:24.0,NON-COUNT,NON-COUNT,SPINAL CORD INJURY,MEDICAL OBSERVATION,abnormal levels of other serum enzymes,other acute ischemic heart diseases,Regular,NaN,Yes,NaN,No,Idaho,R748,abnormal levels of other serum enzymes
3,4,155,100694,88.611203,2020-09-20 09:28:13.0,2020-09-22 11:36:18.0,NON-COUNT,NON-COUNT,SPINAL CORD INJURY,MEDICAL OBSERVATION,viral intestinal infection unspecified,hypo-osmolality and hyponatremia,Regular,NaN,Yes,NaN,No,Idaho,A084,viral intestinal infection unspecified
4,5,322,101407,88.925931,2009-05-03 10:15:50.0,2009-05-04 13:05:08.0,MEDICINE,MEDICINE,SPINAL CORD INJURY OBSERVATION,GENERAL(ACUTE MEDICINE),unspecified dementia,hypertensive chronic kidney disease with stage...,Regular,NaN,Yes,NaN,No,Louisiana,NaN,NaN


In [11]:
# Examining null counts in merged DF
merged_df[['first_dx','second_dx','code','description']].isnull().sum()

first_dx          0
second_dx         0
code           1845
description    1845
dtype: int64

In [12]:
# Comparing sizes of original inpatient_admissions DF and the merged DF
inpt_adm.shape, merged_df.shape

((4010, 18), (4306, 20))

#### Extra Rows
The merged DF has 196 additional rows from the original inpt_adm DF. This is likely due to the duplicate descriptions in the icd_codes DF.

In [13]:
# Comparing patient admission counts between original and merged DF
inpt_adm.Internalpatientid.value_counts()[:10], merged_df.Internalpatientid.value_counts()[:10]

(76555     89
 135216    68
 84413     57
 43802     40
 118613    36
 88033     35
 52767     34
 56635     33
 131776    32
 26517     32
 Name: Internalpatientid, dtype: int64,
 76555     127
 135216     75
 84413      57
 4177       43
 118613     42
 43802      40
 90028      38
 88033      35
 50323      34
 52767      34
 Name: Internalpatientid, dtype: int64)

The number of admissions for the top appearing patients have increased after merging.

Ex: Patient 76555 now has 127 entries instead of the original 89.

### Examining diagnoses without matching ICD codes

In [14]:
# Counts of top 10 first_dx without matching ICD codes
merged_df[merged_df.code.isnull()].first_dx.value_counts()[:10]

atherosclerotic heart disease of native coronary artery                                                        143
alcohol dependence                                                                                             118
unspecified atrial fibrillation and atrial flutter                                                              98
systolic (congestive) heart failure                                                                             66
type 2 diabetes mellitus with other specified complications                                                     61
cutaneous abscess furuncle and carbuncle of limb ; cellulitis and acute lymphangitis of other parts of limb     48
diastolic (congestive) heart failure                                                                            42
post-traumatic stress disorder (ptsd)                                                                           37
cellulitis and acute lymphangitis of other parts of limb                        

In [15]:
# Writing unmatched diagnoses to CSV
# Change filepath and remove block quotes to write csv

"""
unmatched_dx = pd.DataFrame(data = {
    "Unmatched Dx":merged_df[merged_df.code.isnull()].first_dx.value_counts().index,
    "Count":merged_df[merged_df.code.isnull()].first_dx.value_counts().values})
unmatched_dx.to_csv("/content/drive/MyDrive/Colab Notebooks/v-CHAMPS/unmatched_Dx.csv", index = False)

"""

'\nunmatched_dx = pd.DataFrame(data = {\n    "Unmatched Dx":merged_df[merged_df.code.isnull()].first_dx.value_counts().index, \n    "Count":merged_df[merged_df.code.isnull()].first_dx.value_counts().values})\nunmatched_dx.to_csv("/content/drive/MyDrive/Colab Notebooks/v-CHAMPS/unmatched_Dx.csv", index = False)\n\n'

### Why are certain diagnoses unmatched?

Examining further, we can see that the descriptions for ICD codes are more specific than the diagnoses listed.

In [16]:
# Every ICD description concerning coronary atherosclerosis
[desc for desc in icd_codes[icd_codes.description.str.contains("atherosclerotic")].description]

['atherosclerotic heart disease of native coronary artery without angina pectoris',
 'atherosclerotic heart disease of native coronary artery with unstable angina pectoris',
 'atherosclerotic heart disease of native coronary artery with angina pectoris with documented spasm',
 'atherosclerotic heart disease of native coronary artery with other forms of angina pectoris',
 'atherosclerotic heart disease of native coronary artery with unspecified angina pectoris']

In [17]:
# The same as above, for alcohol dependence
[desc for desc in icd_codes[icd_codes.description.str.contains("alcohol dependence")].description]

['alcohol dependence uncomplicated',
 'alcohol dependence in remission',
 'alcohol dependence with intoxication uncomplicated',
 'alcohol dependence with intoxication delirium',
 'alcohol dependence with intoxication unspecified',
 'alcohol dependence with withdrawal uncomplicated',
 'alcohol dependence with withdrawal delirium',
 'alcohol dependence with withdrawal with perceptual disturbance',
 'alcohol dependence with withdrawal unspecified',
 'alcohol dependence with alcohol-induced mood disorder',
 'alcohol dependence with alcohol-induced psychotic disorder with delusions',
 'alcohol dependence with alcohol-induced psychotic disorder with hallucinations',
 'alcohol dependence with alcohol-induced psychotic disorder unspecified',
 'alcohol dependence with alcohol-induced persisting amnestic disorder',
 'alcohol dependence with alcohol-induced persisting dementia',
 'alcohol dependence with alcohol-induced anxiety disorder',
 'alcohol dependence with alcohol-induced sexual dysfuncti

In [18]:
# The same as above, for systolic (congestive) heart failure
[desc for desc in icd_codes[icd_codes.description.str.contains("systolic (congestive) heart", regex = False)].description]

['unspecified systolic (congestive) heart failure',
 'acute systolic (congestive) heart failure',
 'chronic systolic (congestive) heart failure',
 'acute on chronic systolic (congestive) heart failure']

In [19]:
# The same as above, for atrial fibrillation
[desc for desc in icd_codes[icd_codes.description.str.contains("atrial fibrillation")].description]

['paroxysmal atrial fibrillation',
 'persistent atrial fibrillation',
 'chronic atrial fibrillation',
 'unspecified atrial fibrillation']

In [20]:
# The same as above, for type 2 diabetes
[desc for desc in icd_codes[icd_codes.description.str.contains("type 2 diabetes mellitus with other")].description]

['type 2 diabetes mellitus with other diabetic kidney complication',
 'type 2 diabetes mellitus with other diabetic ophthalmic complication',
 'type 2 diabetes mellitus with other diabetic neurological complication',
 'type 2 diabetes mellitus with other circulatory complications',
 'type 2 diabetes mellitus with other diabetic arthropathy',
 'type 2 diabetes mellitus with other skin ulcer',
 'type 2 diabetes mellitus with other skin complications',
 'type 2 diabetes mellitus with other oral complications',
 'type 2 diabetes mellitus with other specified complication']

As for the mismatched "type 2 diabetes..." diagnoses, this is a case of the word 'complication' being singular in the icd_codes and plural in the diagnoses.

## Back to Duplicate ICD descriptions
A closer look shows almost one-third (5096 / >15k) of the total ICD duplicates are 'unspecified' diagnoses.

In [21]:
icd_codes[icd_codes.duplicated("description") & icd_codes.description.str.contains("unspecified")]

,code,description
918,B6790,echinococcosis unspecified
1066,C009,malignant neoplasm of lip unspecified
1790,C8191,hodgkin lymphoma unspecified
1791,C8192,hodgkin lymphoma unspecified
1792,C8193,hodgkin lymphoma unspecified
...,...,...
69917,Y3790XS,military operations unspecified
70764,Z3491,encounter for supervision of normal pregnancy ...
70765,Z3492,encounter for supervision of normal pregnancy ...
70766,Z3493,encounter for supervision of normal pregnancy ...


### Reformatting ICD descriptions and Dropping Duplicates
First, I will examine other occurances of the singular 'complication' in the ICD codes and change them to plural. Then, I will re-merge to see if there is any significant difference.

In [22]:
[desc for desc in icd_codes[icd_codes.description.str.contains("with other specified complication")].description]

['diabetes mellitus due to underlying condition with other specified complication',
 'drug or chemical induced diabetes mellitus with other specified complication',
 'type 1 diabetes mellitus with other specified complication',
 'type 2 diabetes mellitus with other specified complication',
 'other specified diabetes mellitus with other specified complication']

In [23]:
icd_codes.description = [desc.replace("with other specified complication","with other specified complications") for desc in icd_codes.description]

In [24]:
# Merging inpatient_admissions and the formatted ICD codes/descriptions
merged_df2 = pd.merge(inpt_adm, icd_codes, left_on = 'first_dx', right_on = "description", how = 'left')
print(merged_df2.shape)
merged_df2.head(5)

(4306, 20)


,Unnamed: 0,X,Internalpatientid,Age.at.admission,Admission.date,Discharge.date,Admitting.unit.service,Discharging.unit.service,Admitting.specialty,Discharging.specialty,first_dx,second_dx,Discharge.disposition,Died.during.admission,Outpatientreferralflag,Serviceconnectedflag,Agentorangeflag,State,code,description
0,1,4,100012,55.317020,2004-10-25 08:54:01.0,2004-10-26 08:05:06.0,SURGERY,SURGERY,NEUROSURGERY,GENERAL SURGERY,other and unspecified noninfective gastroenter...,other specified disorders of white blood cells,Regular,NaN,Yes,NaN,NaN,New Mexico,NaN,NaN
1,2,82,100399,85.706740,2010-03-24 19:31:38.0,2012-11-10 19:50:48.0,NHCU,NHCU,DOMICILIARY,NH HOSPICE,unspecified mental disorder due to known physi...,malignant neoplasm of stomach unspecified,Death without autopsy,NaN,NaN,No,No,Minnesota,F09,unspecified mental disorder due to known physi...
2,3,154,100694,83.926120,2016-01-12 23:55:24.0,2016-01-13 20:55:24.0,NON-COUNT,NON-COUNT,SPINAL CORD INJURY,MEDICAL OBSERVATION,abnormal levels of other serum enzymes,other acute ischemic heart diseases,Regular,NaN,Yes,NaN,No,Idaho,R748,abnormal levels of other serum enzymes
3,4,155,100694,88.611203,2020-09-20 09:28:13.0,2020-09-22 11:36:18.0,NON-COUNT,NON-COUNT,SPINAL CORD INJURY,MEDICAL OBSERVATION,viral intestinal infection unspecified,hypo-osmolality and hyponatremia,Regular,NaN,Yes,NaN,No,Idaho,A084,viral intestinal infection unspecified
4,5,322,101407,88.925931,2009-05-03 10:15:50.0,2009-05-04 13:05:08.0,MEDICINE,MEDICINE,SPINAL CORD INJURY OBSERVATION,GENERAL(ACUTE MEDICINE),unspecified dementia,hypertensive chronic kidney disease with stage...,Regular,NaN,Yes,NaN,No,Louisiana,NaN,NaN


In [25]:
merged_df2[['first_dx','second_dx','code','description']].isnull().sum()

first_dx          0
second_dx         0
code           1782
description    1782
dtype: int64

#### Dropping Duplicates

There is not a significant difference in the amount of unmatched diagnoses so far. I will drop duplicates of descriptions from the ICD codes, keeping the first occurance of each duplicate, and re-merge.

In [26]:
# Comparing the sizes of icd_codes before and after dropping duplicates
icd_codes.shape, icd_codes.drop_duplicates("description", keep = 'first').shape

((71704, 2), (56553, 2))

In [27]:
# Dropping duplicates and merging
icd_codes = icd_codes.drop_duplicates("description", keep = "first")
merged_dropped_df = pd.merge(inpt_adm, icd_codes, left_on = 'first_dx', right_on = "description", how = 'left')
merged_dropped_df.head(5)

,Unnamed: 0,X,Internalpatientid,Age.at.admission,Admission.date,Discharge.date,Admitting.unit.service,Discharging.unit.service,Admitting.specialty,Discharging.specialty,first_dx,second_dx,Discharge.disposition,Died.during.admission,Outpatientreferralflag,Serviceconnectedflag,Agentorangeflag,State,code,description
0,1,4,100012,55.317020,2004-10-25 08:54:01.0,2004-10-26 08:05:06.0,SURGERY,SURGERY,NEUROSURGERY,GENERAL SURGERY,other and unspecified noninfective gastroenter...,other specified disorders of white blood cells,Regular,NaN,Yes,NaN,NaN,New Mexico,NaN,NaN
1,2,82,100399,85.706740,2010-03-24 19:31:38.0,2012-11-10 19:50:48.0,NHCU,NHCU,DOMICILIARY,NH HOSPICE,unspecified mental disorder due to known physi...,malignant neoplasm of stomach unspecified,Death without autopsy,NaN,NaN,No,No,Minnesota,F09,unspecified mental disorder due to known physi...
2,3,154,100694,83.926120,2016-01-12 23:55:24.0,2016-01-13 20:55:24.0,NON-COUNT,NON-COUNT,SPINAL CORD INJURY,MEDICAL OBSERVATION,abnormal levels of other serum enzymes,other acute ischemic heart diseases,Regular,NaN,Yes,NaN,No,Idaho,R748,abnormal levels of other serum enzymes
3,4,155,100694,88.611203,2020-09-20 09:28:13.0,2020-09-22 11:36:18.0,NON-COUNT,NON-COUNT,SPINAL CORD INJURY,MEDICAL OBSERVATION,viral intestinal infection unspecified,hypo-osmolality and hyponatremia,Regular,NaN,Yes,NaN,No,Idaho,A084,viral intestinal infection unspecified
4,5,322,101407,88.925931,2009-05-03 10:15:50.0,2009-05-04 13:05:08.0,MEDICINE,MEDICINE,SPINAL CORD INJURY OBSERVATION,GENERAL(ACUTE MEDICINE),unspecified dementia,hypertensive chronic kidney disease with stage...,Regular,NaN,Yes,NaN,No,Louisiana,NaN,NaN


In [28]:
# Comparing the shape of the newly merged DF and the inpt_adm DF
merged_dropped_df.shape, inpt_adm.shape

((4010, 20), (4010, 18))

In [29]:
# Comparing Unique patients in each DF
merged_dropped_df.Internalpatientid.nunique(), inpt_adm.Internalpatientid.nunique()

(632, 632)

In [30]:
# Examining nulls in newly merged DF
merged_dropped_df[['first_dx','second_dx','code','description']].isnull().sum()

first_dx          0
second_dx         0
code           1782
description    1782
dtype: int64

In [31]:
# Examining top 10 unmatched diagnoses in merged_dropped_df
merged_dropped_df[merged_dropped_df.code.isnull()].first_dx.value_counts()[:10]

atherosclerotic heart disease of native coronary artery                                                        143
alcohol dependence                                                                                             118
unspecified atrial fibrillation and atrial flutter                                                              98
systolic (congestive) heart failure                                                                             66
cutaneous abscess furuncle and carbuncle of limb ; cellulitis and acute lymphangitis of other parts of limb     48
diastolic (congestive) heart failure                                                                            42
post-traumatic stress disorder (ptsd)                                                                           37
cellulitis and acute lymphangitis of other parts of limb                                                        30
acute pancreatitis                                                              

The issue of the difference in specificity between the ICD codes and the first_dx continues to exist. However, there are entries for second_dx that might be able to be used to clarify the first_dx.

For example, there are patients with an unspecified first_dx of "atherosclerotic heart disease of native coronary artery", but who have second_dx of "unstable angina" etc.

In [42]:
merged_dropped_df[merged_dropped_df.first_dx.str.contains("atherosclerotic heart") & merged_dropped_df.second_dx.str.contains("angina")].head()

,Unnamed: 0,X,Internalpatientid,Age.at.admission,Admission.date,Discharge.date,Admitting.unit.service,Discharging.unit.service,Admitting.specialty,Discharging.specialty,first_dx,second_dx,Discharge.disposition,Died.during.admission,Outpatientreferralflag,Serviceconnectedflag,Agentorangeflag,State,code,description
11,12,1215,105425,53.991401,2004-05-14 20:34:18.0,2004-05-21 13:23:09.0,SURGERY,SURGERY,GRECC-GEM-REHAB,THORACIC SURGERY,atherosclerotic heart disease of native corona...,unstable angina,Regular,NaN,Yes,NaN,No,Missouri,NaN,NaN
152,153,26160,102468,60.406117,2011-12-31 10:27:27.0,2012-01-01 20:57:03.0,MEDICINE,MEDICINE,SUBSTANCE ABUSE INTERMED CARE,MEDICAL ICU,atherosclerotic heart disease of native corona...,unstable angina,Regular,NaN,Yes,NaN,Yes,North Carolina,NaN,NaN
243,244,38653,107797,64.188778,2012-10-28 23:57:37.0,2012-10-30 00:03:39.0,NON-COUNT,NON-COUNT,NEUROLOGY OBSERVATION,MEDICAL OBSERVATION,atherosclerotic heart disease of native corona...,other forms of angina pectoris,Regular,NaN,Yes,NaN,No,Ohio,NaN,NaN
309,310,45321,105907,65.997269,2014-06-26 00:02:42.0,2014-07-01 02:10:18.0,MEDICINE,MEDICINE,PERIPHERAL VASCULAR,Not specified (no value),atherosclerotic heart disease of native corona...,unstable angina,Regular,NaN,Yes,NaN,No,Alabama,NaN,NaN
418,419,72306,115675,74.390507,2000-06-26 17:31:35.0,2000-06-26 23:49:36.0,MEDICINE,MEDICINE,HIGH INTENSITY GEN PSYCH INPAT,MEDICAL ICU,atherosclerotic heart disease of native corona...,unstable angina,Transfer,NaN,No,No,NaN,Ohio,NaN,NaN
